# Bayesian Analysis Agent

In [ ]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage
from langchain_mcp_adapters.client import MultiServerMCPClient, StreamableHttpConnection
from langgraph.checkpoint.memory import InMemorySaver

from chain_reaction.config import APIKeys, ModelBehavior, ModelName
from chain_reaction.utils import get_last_message

# Get tools and prompt from MCP sever

In [ ]:
# Start MCP sever
uv run fastmcp run 3-mcp-fundamentals/1-bayesian-tools/server.py:mcp --transport http --port 8000

In [ ]:
# Initialize a client connecting to the Bayesian tools MCP server
client = MultiServerMCPClient(
    {
        "bayesian-tools": StreamableHttpConnection(url="http://localhost:8000/mcp", transport="streamable_http"),
    },
)

In [ ]:
# Retrieve tools and system prompt from the Bayesian tools MCP server
tools = await client.get_tools(server_name="bayesian-tools")
prompt_messages = await client.get_prompt(server_name="bayesian-tools", prompt_name="system_prompt")
system_prompt = prompt_messages[0].content

# Initialize agent

In [ ]:
# Create a LangChain agent using the retrieved tools and system prompt
bayesian_agent = create_agent(
    model=init_chat_model(
        model=ModelName.CLAUDE_HAIKU,
        timeout=None,
        max_retries=2,
        api_key=APIKeys().anthropic,
        **ModelBehavior.factual().model_dump(),
    ),
    checkpointer=InMemorySaver(),
    tools=tools,
    system_prompt=system_prompt,
)

# Multi-turn conversation

In [ ]:
# Initialize configuration for a new chat session
config = {"configurable": {"thread_id": 1}}

## Turn 1

In [ ]:
# NOTE: Using `ainvoke` for async call b/c MCP tools are async
response = await bayesian_agent.ainvoke(
    {
        "messages": [
            HumanMessage(content="I'm trying to figure out the likelihood of rain tomorrow, can you help me out?")
        ]
    },
    config=config,
)
print(get_last_message(response).content)

**Example response**

---

I'd be happy to help you estimate the likelihood of rain tomorrow using Bayesian modeling! This is a great use case for beta-binomial analysis.

To get started, I need to understand your prior beliefs about rain in your area. Let me ask you a few questions:

1. **What's your prior estimate?** Based on your experience and knowledge of the weather patterns in your location, what probability would you assign to it raining tomorrow? (For example, 0.3 for 30%, 0.5 for 50%, etc.)

2. **How confident are you in this estimate?** I can help you express your conviction level using one of these categories:
   - **Random**: You have no real basis for your estimate
   - **Gut feeling**: You have a vague sense but no strong evidence
   - **Educated guess**: You have some experience or data to back it up
   - **Reasonably confident**: You have good historical data or knowledge
   - **Very confident**: You have extensive data or expertise
   - **Bet the farm**: You're extremely certain about this estimate

3. **Do you have any recent weather data?** For example, have you checked a weather forecast, or do you have historical data about how many days it rained in your area over a certain period?

Once you provide these details, I can help you:
- Set up your prior belief as a beta distribution
- Update it with any observed data you have
- Calculate confidence intervals for your rain probability estimate
- Generate sample predictions for tomorrow

What are your thoughts on these questions?

## Turn 2

In [ ]:
response = await bayesian_agent.ainvoke(
    {
        "messages": [
            HumanMessage(content="I believe there's a 1% chance of rain tomorrow, but this is mostly a gut feeling.")
        ]
    },
    config=config,
)
print(get_last_message(response).content)

**Example response**

---

Perfect! Based on your prior belief of 1% chance of rain with a "gut feeling" conviction level, your beta distribution parameters are:
- **Alpha (α) = 0.07** (representing "successes" or rainy days)
- **Beta (β) = 6.93** (representing "failures" or non-rainy days)

This reflects your belief that rain is quite unlikely, but with relatively low conviction since it's just a gut feeling.

Now, a few follow-up questions to help refine this further:

1. **Do you have any observed data?** For example:
   - Have you checked a weather forecast that gives a specific probability?
   - Do you have historical data about how many days it rained in your area over the past week, month, or season?

2. **Would you like me to:**
   - Calculate a confidence interval around your 1% estimate?
   - Generate sample predictions to see the range of possible outcomes?
   - Help you update your belief if you get new information (like a weather forecast)?

What would be most helpful for you?

## Turn 3

In [ ]:
response = await bayesian_agent.ainvoke(
    {
        "messages": [
            HumanMessage(
                content="""I just checked some historical data, and for the last 5 years in January, it's rained on
                average 7.4 out of 31 days (37 rainy days out of 155 observed days)."""
            )
        ]
    },
    config=config,
)
print(get_last_message(response).content)

**Example Response**

---

Great! Here's what your updated belief looks like after incorporating the historical data:

**Updated Beta Distribution:**
- **Alpha (α) = 37.07** (rainy days)
- **Beta (β) = 124.93** (non-rainy days)

**Key Insights:**

1. **Estimated Probability of Rain Tomorrow:** ~23.8% (37.07 / (37.07 + 124.93))
   - This is much higher than your initial 1% gut feeling!

2. **95% Confidence Interval:** 16.8% to 29.6%
   - This means we can be 95% confident that the true probability of rain in January falls somewhere between these bounds.

**What This Means:**
Your historical data has significantly updated your belief. The data shows that January in your area is actually fairly rainy, with about 1 in 4 days experiencing rain. Your initial gut feeling of 1% was quite optimistic compared to the historical pattern!

**Next Steps:**
- Would you like me to generate sample predictions to see the range of possible outcomes?
- Do you have any additional information (like a weather forecast) that might further refine this estimate?
- Would you like to adjust your conviction level in the historical data?

## Turn 4

In [ ]:
response = await bayesian_agent.ainvoke(
    {
        "messages": [
            HumanMessage(
                content="""There are 27 days left in January; can you simulate what the outcome for rain might be
                this month based on everything we've discussed so far?"""
            )
        ]
    },
    config=config,
)
print(get_last_message(response).content)

**Example Response**

---

Perfect! I've simulated 10,000 possible outcomes for the remaining 27 days of January based on your updated belief. Here's what the simulation tells us:

**Summary of Simulation Results:**

**Expected Number of Rainy Days in the Remaining 27 Days:**
- **Average probability of rain per day:** ~22.9%
- **Expected rainy days:** About 6-7 days out of the remaining 27 days
- **Most likely range:** 4-9 rainy days

**Key Findings:**

1. **Probability Distribution:** The simulation shows that rainy days are fairly likely to occur, with the distribution centered around 6-7 rainy days for the rest of January.

2. **Confidence Range:** Based on the simulation, there's a good chance you'll see somewhere between 4-9 rainy days in the remaining 27 days of January.

3. **Comparison to Your Initial Belief:** Your original gut feeling of 1% rain was quite optimistic! The historical data shows that January in your area is actually fairly wet, with roughly 1 in 4 days experiencing rain.

**What This Means for Tomorrow:**

Based on everything we've discussed, **tomorrow has approximately a 23% chance of rain** - significantly higher than your initial 1% estimate, but still more likely to be dry than rainy.

Would you like me to:
- Calculate the probability for specific scenarios (e.g., "What's the chance of 5+ rainy days in the remaining month?")?
- Help you update this forecast if you get new information (like a weather forecast)?
- Explore how confident you should be in these estimates?